In [16]:
import re

anabtb = {}
resptb = {}
word2sent = {}

def normlemma(curlemma):
    curlemma = re.sub('-','',curlemma)
    curlemma = re.sub('ῑ','ι',curlemma)
    curlemma = re.sub('ᾱ','α',curlemma)
    curlemma = re.sub('ῡ','υ',curlemma)
    return(curlemma)

pharrvoc = {}
curlesson = 0
curlemma = ''
f = open('/Users/gcrane/github/CrosbySchaeffer2.0/crosbycshaeffer2.0.xml')
for l in f:
    continue
    m = re.search('subtype="lesson" n="([0-9]+)"',l)
    if(m):
        curlesson = int(m[1])
        #print(curlesson)
        
    if(curlesson > 34):
        continue
        
    m = re.search('<rs type="lemma"([^>]*)>([^<]+)',l)
    if(m):
        if(m[1]):
            curlemma = re.sub('\s+n="([^"]+)"','\g<1>',m[1])
            curlemma = normlemma(curlemma)
            print(curlesson,curlemma)
        else:
            curlemma = m[2]
            curlemma = normlemma(curlemma)
        #curlemma = re.sub('-','',curlemma)
        #curlemma = re.sub('ῑ','ι',curlemma)
        #curlemma = re.sub('ᾱ','α',curlemma)
        #curlemma = re.sub('ῡ','υ',curlemma)
    if(curlemma):
        if(curlemma in pharrvoc ):
            pharrvoc[curlemma].append(str(curlesson))
        else:
            pharrvoc[curlemma] = [str(curlesson)]
        curlemma = ''
        

f.close()
# collect short definitions for vocabulary
lexicalentries = {}
verbarguments = {}
lexfile = "/Users/gcrane/github/Thucydides-new-working-materials/GreekMorphApril18.csv"
with open(lexfile) as f:
    for line in f:

        #print(line)
        #line = line.strip('"')
        fields = line.split(",")
        fields[1] = fields[1].strip('"')
        fields[2] = fields[2].strip('\n')
        fields[2] = fields[2].strip('"')
        lexicalentries[fields[1]] = fields[2]

f.close()

resptbname = "0059-030.xml"
anabtbname = '0032-006.xml'


anabvoc = {}
respvoc = {}
anabpos = {}
resppos = {}
totpos = {}

resptokens = 0
anabtokens = 0
resptokens1 = 0
anabtokens1 = 0
anabverbs = {}
respverbs = {}
totverbs = {}
tottokens = 0

anabvoc1 = {}
respvoc1 = {}

totvoc = {}

def additem(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1

def countvoc(tbname,fulltb,subwork,fullwork,fullvoc,fullpos,totpos,curverbs,totverbs,subpat):
    f = open(tbname)
    subtoks = 0
    tottoks = 0
    global word2sent
    
    spat = 'subdoc="' + subpat
    for l in f:
        l = re.sub('\n','',l)
        m = re.search('<sentence id="([^"]+)"',l)
        if(m):
            cursent = m[1]
            if(re.search(spat,l)):
                insub = 1
            else:
                insub = 0

        if(0 and re.search('postag="u',l)):
            continue
            
        m = re.search('word id="([^"]+)"',l)
        if(m):
            wordid = m[1]
            word2sent[wordid]  = cursent
            
        m = re.search('subdoc="([^"]+)"',l)
        if(m):
            curcit = m[1]
        m = re.search('lemma="([^"]+)"\s+postag="(.)(..(...)[^"]+)',l)
        if(m):
            fulltb[l] = curcit
            curlemma = m[1] + '\t' + m[2]
            curpos = m[2] + m[3]
            vparad = m[4]
            additem(curlemma,fullvoc)
            if(m[2] == 'v'):
                additem(vparad,totverbs)
                additem(vparad,curverbs)
            additem(curlemma,fullwork)
            additem(curpos,fullpos)
            additem(curpos,totpos)
            tottoks = tottoks + 1
            
            if(insub):
                additem(curlemma,subwork)
                subtoks = subtoks + 1
               
        
        
    
    f.close()
    return tottoks,subtoks


resppat = '(32[789]|3[34][0-9]|35[0-4])'
resppat = '(32[789]|33[01])'
anabpat = '1\.'

resptokens,resptokens1 = countvoc(resptbname,resptb,respvoc1,respvoc,totvoc,resppos,totpos,respverbs,totverbs,resppat)

anabtokens,anabtokens1 = countvoc(anabtbname,anabtb,anabvoc1,anabvoc,totvoc,anabpos,totpos,anabverbs,totverbs,anabpat)


In [17]:
 pharrvoc['οὕτως']

KeyError: 'οὕτως'

In [18]:
i = 0

outf = open('respanabv.tsv','w')
for foo in sorted(totvoc,key=totvoc.get,reverse=True):
    i = i + 1
    curlem = foo.split()[0]
    print('foo',curlem,curlem in pharrvoc)
    if(curlem in pharrvoc):
        pharrnum = pharrvoc[curlem][0]
        print(curlem,pharrnum)
    else:
        pharrnum = 'na'
    if(i>1000):
        continue
    if(not foo in anabvoc1):
        anab1n = 0
    else:
        anab1n = re.sub('(\.[0-9][0-9]).+','\g<1>',str((anabvoc1[foo]/anabtokens1)*10000))
    if(not foo in respvoc1):
        resp1n = 0
    else:
        resp1n = re.sub('(\.[0-9][0-9]).+','\g<1>',str((respvoc1[foo]/resptokens1)*10000))
        
    baselem = foo.split()[0]
    if(baselem in lexicalentries):
        curdef = lexicalentries[baselem]
    else:
        curdef = 'nodef'
        
    totfreq = re.sub('(\.[0-9][0-9]).+','\g<1>',str(totvoc[foo]/(resptokens+anabtokens)*10000))

    print(i,foo,curdef,pharrnum,totfreq,anab1n,resp1n,sep='\t')
    print(i,foo,curdef,pharrnum,totfreq,anab1n,resp1n,sep='\t',file=outf)
    
outf.close()

foo ὁ False
1	ὁ	l	the	na	866.45	1007.71	833.78
foo , False
2	,	u	nodef	na	793.67	521.00	877.09
foo καί False
3	καί	b	and	na	508.19	536.24	503.51
foo . False
4	.	u	nodef	na	354.31	355.27	378.99
foo δέ False
5	δέ	b	but	na	290.37	385.75	194.91
foo εἰμί False
6	εἰμί	v	to be	na	238.09	173.34	249.05
foo οὗτος False
7	οὗτος	a	this; that	na	147.68	161.92	113.69
foo · False
8	·	u	nodef	na	135.37	217.16	157.01
foo αὐτός False
9	αὐτός	a	unemph. 3rd pers.pronoun; -self; [the] same 	na	135.25	172.39	124.52
foo ἐγώ False
10	ἐγώ	p	I (first person pronoun)	na	132.19	73.34	238.22
foo ; False
11	;	u	nodef	na	111.32	8.57	86.62
foo τε False
12	τε	b	and	na	106.49	45.71	97.45
foo μέν False
13	μέν	d	on the one hand	na	106.14	96.19	54.14
foo οὐ False
14	οὐ	d	not	na	102.95	62.86	97.45
foo φημί False
15	φημί	v	to say	na	98.42	30.47	124.52
foo λέγω False
16	λέγω	v	to pick; to say	na	97.24	70.48	162.42
foo ὅς False
17	ὅς	p	who	na	94.58	85.72	108.28
foo τις False
18	τις	a	any one	na	85.69	68.57	151.59
foo ἐν False

foo χρή False
260	χρή	v	it is fated	na	4.06	1.90	37.89
foo ἐπιθυμία False
261	ἐπιθυμία	n	desire	na	4.06	0	10.82
foo ἔνειμι False
262	ἔνειμι	v	to be in; to be possible	na	4.06	3.80	0
foo Τισσαφέρνης False
263	Τισσαφέρνης	n	Tissaphernes	na	4.06	19.04	0
foo ὅμοιος False
264	ὅμοιος	a	like	na	4.00	0	0
foo πειράω False
265	πειράω	v	to attempt	na	3.94	4.76	0
foo νύξ False
266	νύξ	n	the night	na	3.94	1.90	0
foo ἀναγκάζω False
267	ἀναγκάζω	v	to force	na	3.94	0	0
foo ὁπλίτης False
268	ὁπλίτης	n	heavy-armed	na	3.94	19.04	0
foo ἔνθα False
269	ἔνθα	d	there	na	3.94	8.57	0
foo ἄξιος False
270	ἄξιος	a	worthy	na	3.88	4.76	10.82
foo θύω False
271	θύω	v	to sacrifice	na	3.88	1.90	5.41
foo οἰκεῖος False
272	οἰκεῖος	a	in or of the house	na	3.88	0	16.24
foo πότερον False
273	πότερον	d	whether	na	3.88	0.95	16.24
foo ἀναγκαῖος False
274	ἀναγκαῖος	a	necessary	na	3.88	0.95	0
foo φύω False
275	φύω	v	to bring forth	na	3.83	0.95	0
foo ὅλος False
276	ὅλος	a	whole	na	3.83	0.95	0
foo ἴδιος False
277	ἴδιος	a	one's own	

foo συνέρχομαι False
510	συνέρχομαι	v	come together	na	1.76	0.95	10.82
foo ἔμπροσθεν False
511	ἔμπροσθεν	r	before	na	1.76	0.95	5.41
foo δίκαιος False
512	δίκαιος	d	just	na	1.76	0.95	10.82
foo βλάπτω False
513	βλάπτω	v	to disable	na	1.76	0	0
foo σημαίνω False
514	σημαίνω	v	to shew by a sign	na	1.76	0	0
foo διαπράσσω False
515	διαπράσσω	v	to pass over; bring about	na	1.76	0	0
foo προστάσσω False
516	προστάσσω	v	to order	na	1.76	1.90	0
foo πως False
517	πως	d	somehow	na	1.76	0.95	0
foo βία False
518	βία	n	bodily strength	na	1.76	0.95	0
foo ἀπεργάζομαι False
519	ἀπεργάζομαι	v	to finish off	na	1.76	0	0
foo κατέχω False
520	κατέχω	v	to hold fast	na	1.76	0	0
foo γυμναστικός False
521	γυμναστικός	a	fond of athletic exercises	na	1.76	0	0
foo γέλοιος False
522	γέλοιος	a	causing laughter	na	1.76	0	0
foo ἀσφαλής False
523	ἀσφαλής	a	not liable to fall	na	1.76	2.85	0
foo τριάκοντα False
524	τριάκοντα	a	thirty	na	1.76	8.57	0
foo ̔ False
525	̔	u	nodef	na	1.76	4.76	0
foo ̓ False
526	̓	u	nodef	na	1.76	4

800	ἀπαιτέω	v	to demand back	na	1.00	0.95	5.41
foo ἀπιστέω False
801	ἀπιστέω	v	to disbelieve	na	1.00	0	0
foo ἀγνοέω False
802	ἀγνοέω	v	not to perceive	na	1.00	0	0
foo ὠφέλεια False
803	ὠφέλεια	n	help	na	1.00	0	0
foo κυβερνήτης False
804	κυβερνήτης	n	a steersman	na	1.00	0	0
foo νοέω False
805	νοέω	v	to perceive by the eyes	na	1.00	0	0
foo εἴωθα False
806	εἴωθα	v	to be accustomed	na	1.00	0	0
foo εἶτα False
807	εἶτα	d	then	na	1.00	5.71	0
foo ποιέω False
808	ποιέω	a	to make	na	1.00	0	0
foo ἀκριβής False
809	ἀκριβής	a	exact	na	1.00	0	0
foo ἐπιτάσσω False
810	ἐπιτάσσω	v	to put upon one as a duty	na	1.00	0	0
foo πλεονεκτέω False
811	πλεονεκτέω	v	to have or claim more than one’s share; to have an advantage	na	1.00	0	0
foo ἐμβάλλω False
812	ἐμβάλλω	v	to throw in	na	1.00	3.80	0
foo θαρσέω False
813	θαρσέω	v	to be of good courage	na	1.00	0.95	0
foo ὄμμα False
814	ὄμμα	n	the eye	na	1.00	0	0
foo πω False
815	πω	d	up to this time	na	1.00	0.95	0
foo ἁρπάζω False
816	ἁρπάζω	v	to snatch away	na	1.00	3.

foo προσποιέω False
foo δημοκρατικός False
foo ἐπιλείπω False
foo ποιμήν False
foo δουλόω False
foo κήδω False
foo ἐνδείκνυμι False
foo ἐξευρίσκω False
foo ἄφρων False
foo ἐπιστήμων False
foo ἀμαθία False
foo ἀποβαίνω False
foo ἐγκωμιάζω False
foo ἐκφεύγω False
foo δεσμός False
foo διίστημι False
foo παρακελεύομαι False
foo ὗς False
foo ὑπερβαίνω False
foo διακελεύομαι False
foo ὀνίνημι False
foo γυμνός False
foo ἄρτος False
foo ὑγιεινός False
foo χρῶμα False
foo ἀθλητής False
foo ἀλλοιόω False
foo ταράσσω False
foo μίμημα False
foo ἐλπίζω False
foo φοβερός False
foo ἐξαιρέω False
foo δαίμων False
foo διηγέομαι False
foo ἀσφαλής False
foo βίαιος False
foo περιφέρω False
foo ἐμβαίνω False
foo ἐσθίω False
foo ἐμπειρία False
foo μηδαμῇ False
foo προσφέρω False
foo ἀπολογέομαι False
foo διπλάσιος False
foo παίζω False
foo λευκός False
foo προσάγω False
foo ἀντιλέγω False
foo πρόφασις False
foo διασπάω False
foo πένης False
foo νίκη False
foo ἄτιμος False
foo ῥέω False
foo ὑψηλός False
foo 

foo ἐξεργάζομαι False
foo σφέτερος False
foo λύκος False
foo φρουρέω False
foo ἀποδημέω False
foo παρατίθημι False
foo κακουργία False
foo τριπλάσιος False
foo συναθροίζω False
foo προπολεμέω False
foo ἠώς False
foo ἀνωφελής False
foo Δελφοί False
foo πάτριος False
foo τοῖος False
foo ἄνθος False
foo ῥύμμα False
foo ἀποκάμνω False
foo ἕδρα False
foo πεινέω False
foo ὀρέγω False
foo ἀνέρχομαι False
foo προστρέχω False
foo συνέχω False
foo ἐπιτελέω False
foo ἀρχικός False
foo δειλία False
foo ἀριστοκρατία False
foo διοίκησις False
foo ψῆφος False
foo ὄκνος False
foo φόνος False
foo πορεύω False
foo μάταιος False
foo διοικέω False
foo ἐπίσκεψις False
foo διατάσσω False
foo δριμύς False
foo ἀκμάζω False
foo ἀστασίαστος False
foo νυμφίος False
foo ἐπανέρχομαι False
foo κτῆμα False
foo πανταχῇ False
foo ἀσφάλεια False
foo ἐπικίνδυνος False
foo φείδομαι False
foo σωφρονίζω False
foo δυνάστης False
foo φιλοσοφέω False
foo παῦλα False
foo στέργω False
foo δάκνω False
foo ἐπιθυμητής False
foo φι

foo αἰσχροκερδής False
foo ὑγιής False
foo δικαστικός False
foo ἀγριότης False
foo ἄγροικος False
foo καταυλέω False
foo νεῦρον False
foo γεωργικός False
foo κηδεμών False
foo τραγικός False
foo μεταδοξάζω False
foo τάφος False
foo τόλμα False
foo ὄνειρος False
foo γηγενής False
foo ἐτός False
foo φυή False
foo χρησμός False
foo φήμη False
foo μιαίνω False
foo ὄροφος False
foo οἰκονόμος False
foo ἐπιθυμόω False
foo κεραμεύω False
foo νουθετέω False
foo ἄρδην False
foo χυτρεύς False
foo παραδύομαι False
foo τρυφή False
foo πίων False
foo πνῖγος False
foo χειρόω False
foo ἰσχνός False
foo ἀφορίζω False
foo παίδευσις False
foo νεωτερίζω False
foo εὐλαβέομαι False
foo ἀνάτρεψις False
foo εἰσδέχομαι False
foo σχηματισμός False
foo αἰκία False
foo προαγορεύω False
foo νομοθεσία False
foo θήκη False
foo ἐξηγητής False
foo ὀμφαλός False
foo ὑπόλοιπος False
foo κατάδηλος False
foo προσρητέος False
foo δευσοποιός False
foo φ́ω False
foo δύσβατος False
foo σκοτεινός False
foo ἰού False
foo δύσκρι

foo μυθοποιός False
foo ἐκβλητέον False
foo Οὐρανός False
foo γιγαντομαχία False
foo μυθολογητέον False
foo ποικιλτέον False
foo ἀπέχθομαι False
foo λογοποιέω False
foo θεομαχία False
foo δυσέκνιπτος False
foo ἐπιτρεπτέον False
foo θεολογία False
foo εὐπραγία False
foo αἰτιατέον False
foo δοιοί False
foo πίθος False
foo οὖδας False
foo βούβρωστις False
foo δῖος False
foo ταμίας False
foo τεύχω False
foo σύγχυσις False
foo Θέμις False
foo παμπήδην False
foo Νιόβη False
foo Πελοπίδης False
foo Τρωικός False
foo ἐξευρετέος False
foo διαμαχετέον False
foo σύμψηφος False
foo ἀμφίεσμα False
foo ἀλλοδαπός False
foo ἐπιστρωφάω False
foo Πρωτεύς False
foo Ἴναχος False
foo βιόδωρος False
foo ἐκδειματόω False
foo ἰνδάλλομαι False
foo βλασφημέω False
foo ἀποτροπή False
foo φαντασία False
foo ψεῦδις False
foo ἐνδατέομαι False
foo εὐπαιδία False
foo μακραίων False
foo παιάν False
foo ἐπευφημέω False
foo Φοῖβος False
foo μαντικός False
foo βρύον False
foo θοίνη False
foo θεοσεβής False
foo δεῖνα Fals

foo ἀκούσιος False
foo εὔκολος False
foo ἀμαθαίνω False
foo ὕειος False
foo μολύνω False
foo διευλαβητέον False
foo ἀρτιμελής False
foo ἀρτίφρων False
foo γηράσκω False
foo προπαιδεία False
foo προπαιδεύω False
foo ἐπανάγκης False
foo πονέομαι False
foo ἔμμονος False
foo προσάγω False
foo γευστέον False
foo ἐντρεχής False
foo βάσανος False
foo εἰκοσιετής False
foo χύδην False
foo συνακτέον False
foo σύνοψις False
foo συνοπτικός False
foo πρόκριτος False
foo ὑποβολιμαῖος False
foo ὑποβάλλω False
foo ὑποβολή False
foo ἀπαρακάλυπτος False
foo τῇδε False
foo πειθαρχία False
foo δόξις False
foo τριακονταετής False
foo μειρακίσκος False
foo παιδιή False
foo σκυλάκιον False
foo σπαράσσω False
foo στάσιμος False
foo μετάληψις False
foo ἐνδελεχής False
foo ἀντίστροφος False
foo καταβιβαστέος False
foo νεάω False
foo πανταχόσε False
foo πεντηκοντούτης False
foo ἀνακλίνω False
foo ἐπιταλαιπωρέω False
foo ἀντικαταλείπω False
foo συναναιρέω False
foo ἀνδριαντοποιός False
foo διασκευωρέω False
foo δ

foo βάσιμος False
foo σκεδάννυμι False
foo ὑπερύψηλος False
foo Ῥόδιος False
foo ἀποδέρω False
foo φυσάω False
foo ἀρτάω False
foo ἄγκυρα False
foo ἐπιφορέω False
foo καταδύνω False
foo ὀλισθάνω False
foo ὑπαναχωρέω False
foo ἄκαυστος False
foo θερίζω False
foo ἀπονοστέω False
foo δυσχωρία False
foo ἐπιμείγνυμι False
foo ἑκασταχόσε False
foo παραβαίνω False
foo ἐμβολή False
foo παράγγελσις False
foo ἄγκος False
foo χάλκωμα False
foo ὑποφείδομαι False
foo διΐημι False
foo κατατιτρώσκω False
foo ἀπροσδόκητος False
foo Καρδοῦχος False
foo εὐπρεπής False
foo στένος False
foo θαμινός False
foo Λεώνυμος False
foo κατασφάζω False
foo δυσπάριτος False
foo ἀντιστασιάζω False
foo ὁλοίτροχος False
foo ἁμαξιαῖος False
foo διασφενδονάω False
foo καταδιώκω False
foo ὁμίχλη False
foo ἀνιμάω False
foo διαζεύγνυμαι False
foo παριόω False
foo Κηφισοφών False
foo Ἀμφίδημος False
foo ἀμαχητί False
foo προσμείγνυμι False
foo ἀντίπορος False
foo κατάγνυμι False
foo ὑπασπιστής False
foo λάκκος False
foo κονι

In [4]:
i = 0


outf = open('respanabvbs.tsv','w')
for foo in sorted(totverbs,key=totverbs.get,reverse=True):
    i = i + 1
    if(i>100):
        continue
    if(not foo in anabverbs):
        anab1n = 0
    else:
        anab1n = re.sub('(\.[0-9][0-9]).+','\g<1>',str((anabverbs[foo]/anabtokens)*10000))
    if(not foo in respverbs):
        resp1n = 0
    else:
        resp1n = re.sub('(\.[0-9][0-9]).+','\g<1>',str((respverbs[foo]/resptokens)*10000))
    totfreq = re.sub('(\.[0-9][0-9]).+','\g<1>',str(totverbs[foo]/(resptokens+anabtokens)*10000))

    print(i,foo,totfreq,anab1n,resp1n)

1 iia 295.76 330.50 273.29
2 pia 266.54 209.27 303.57
3 ppa 232.23 235.12 230.36
4 pna 223.11 194.25 241.77
5 pie 104.62 75.29 123.60
6 aia 102.43 189.01 46.43
7 ppe 97.97 86.47 105.41
8 apa 80.47 132.41 46.88
9 pne 70.66 69.87 71.17
10 ana 57.21 76.86 44.51
11 poa 55.09 63.23 49.82
12 iie 54.33 122.10 10.50
13 psa 44.04 28.99 53.77
14 fim 38.55 35.63 40.44
15 fia 37.73 25.50 45.64
16 ria 30.73 18.16 38.86
17 aim 30.46 60.44 11.07
18 rpe 27.99 33.88 24.17
19 apm 26.07 30.91 22.93
20 asa 25.86 22.36 28.13
21 anm 23.12 24.98 21.91
22 rpa 21.19 18.16 23.16
23 poe 19.89 25.32 16.38
24 aoa 19.07 24.28 15.70
25 pma 17.90 11.18 22.25
26 pse 17.42 14.32 19.43
27 aip 13.99 26.55 5.87
28 app 13.72 13.62 13.78
29 asm 10.42 7.51 12.31
30 rie 9.81 5.24 12.76
31 rna 8.64 6.11 10.28
32 aom 8.57 9.43 8.02
33 fna 8.57 10.65 7.23
34 fnm 7.06 8.55 6.10
35 rne 6.17 3.84 7.68
36 anp 6.10 7.33 5.31
37 fpa 5.83 12.22 1.69
38 ama 5.42 5.93 5.08
39 lia 4.73 9.25 1.80
40 pme 4.59 4.54 4.63
41 fpm 4.32 6.46 2.93

In [5]:
i = 0


outf = open('respanabpos.tsv','w')
for foo in sorted(totpos,key=totpos.get,reverse=True):
    i = i + 1
    if(i>1000):
        continue
    if(foo in anabpos):
        anabposn = anabpos[foo]
    else:
        anabposn = 0

    if(foo in resppos):
        respposn = resppos[foo]
    else:
        respposn = 0
        
    totposn = respposn + anabposn
    
    print(foo,totposn,respposn,anabposn,sep='\t',file=outf)
outf.close()
    

In [6]:
for foo in sorted(respvoc1,key=respvoc1.get,reverse=True):
    print(respvoc1[foo],foo)

154 ὁ	l
93 καί	b
46 εἰμί	v
44 ἐγώ	p
36 δέ	b
30 λέγω	v
28 τις	a
23 φημί	v
23 αὐτός	a
21 οὗτος	a
21 γάρ	d
20 ὅς	p
18 τε	b
18 οὐ	d
17 ἀλλά	b
17 ὦ	i
16 οὖν	d
15 πολύς	a
14 ἤ	b
14 ἄν	d
14 γε	d
12 σύ	p
12 ὡς	c
12 γῆρας	n
11 Σωκράτης	n
11 ὅτι	c
10 μέν	d
10 πρός	r
10 Πολέμαρχος	n
10 μή	d
10 ἔχω	v
10 δή	d
9 ποιέω	v
9 δοκέω	v
8 Κέφαλος	n
8 ἄλλος	a
8 περί	r
7 νῦν	d
7 γίγνομαι	v
7 χρή	v
7 εἰ	c
7 παρά	r
7 εὖ	d
7 οὔτε	b
6 εἰς	r
6 ὅσος	p
6 ἔτι	d
6 οὕτως	d
6 ἔρχομαι	v
6 ἐν	r
6 πάνυ	d
6 κτάομαι	v
6 ἑαυτοῦ	p
5 Γλαύκων	n
5 μέντοι	d
5 διά	r
5 ὥσπερ	c
5 ἤδη	d
5 ἀληθής	a
5 χρῆμα	n
5 ἀποδίδωμι	v
4 μετά	r
4 τίς	a
4 παῖς	n
4 περιμένω	v
4 λαμβάνω	v
4 πῶς	d
4 εἷς	a
4 ἐάν	c
4 πείθω	v
4 πατήρ	n
4 δέω	v
4 λόγος	n
4 χαλεπός	a
4 τοιοῦτος	a
4 πᾶς	a
4 οἴομαι	v
3 θεός	n
3 θεάομαι	v
3 φαίνω	v
3 ἀπέρχομαι	v
3 ἄστυ	n
3 κελεύω	v
3 ἀπό	r
3 ὁράω	v
3 οὐδέ	d
3 οἶδα	v
3 ἵππος	n
3 αὐτόθι	d
3 μάλα	d
3 ἐπί	r
3 ῥᾴδιος	d
3 δεῦρο	d
3 ἡδονή	n
3 οἰκεῖος	a
3 σφόδρα	d
3 ἡλικία	n
3 ποιητής	n
3 πότερον	d
3 βίος	n
3 οἷος	p
3 μέγας	a
3 τότε	d

In [7]:
resptokens1

1568

In [19]:


def getforms(curtb,label,lemmend,parad,cits):
    outfv = open('voc'+ cits + '.tsv','w')
    outf = open(parad + '.tsv','w')
    i = 0
    for foo in curtb:
        if(cits and not re.search(cits,curtb[foo])):
            continue
        
        m = re.search('word id="([^"]+)"',foo)
        if(m):
            curid = m[1]
        m = re.search('form="([^"]+)"',foo)
        if(m):
            curform = m[1]
        m = re.search('relation="([^"]+)"',foo)
        if(m):
            currel = m[1]
        m = re.search('head="([^"]+)"',foo)
        if(m):
            curhead = m[1]
        m = re.search('postag="([^"]+)"',foo)
        if(m):
            curpos = m[1]
        i = i + 1
        
        m = re.search('lemma="([^"]+)"',foo)
        
        if(m):
            curlem = m[1]
            if(curlem in pharrvoc):
                curdef = 'pharr' + pharrvoc[curlem][0] 
            elif(curlem in lexicalentries):
                curdef = lexicalentries[curlem]
            else:
                curdef = 'nodef'
            print(curtb[foo],word2sent[curid],curid,curform,curlem,curdef,curhead,sep='\t',file=outfv)
        else:
            continue
            
        m = re.search(parad,foo)
        if(not m):
            continue
        curparad = m[1]

        lempat = 'lemma="([^"]+' + lemmend + ')"'
        if(not re.search(lempat,foo)):
            continue

        if(not re.search(cits,curtb[foo])):
            continue
            
            
        print(label,curtb[foo],word2sent[curid],curid,curform,curlem,curpos,curdef,curparad,currel,curhead,sep='\t',file=outf)
    outf.close()
    outfv.close()
        
getforms(resptb,'w-stem pia','ω','v(..pia)','(32[789]|33[01])')

getforms(resptb,'os_ou','(ός|ος)','n.(.---[mf].)','(32[789]|33[01])')
getforms(resptb,'os_ou','(όν|ον)','n.(.---[nf].)','(32[789]|33[01])')

In [3]:
getforms(anabtb,'w-stem iia','ω','v(..iia)','^1\.')
getforms(anabtb,'aor iia','.','v(..aia)','^1\.')
getforms(resptb,'perf r','.','v(..r.a)','(32[789]|33[01])')
getforms(resptb,'perf r','.','v(..r.[pe])','(32[789]|33[01])')
getforms(resptb,'aorist passive','.','v(..a.[p])','(32[789]|33[01])')


In [11]:
getforms(resptb,'1st declension','(η|ή|α|ά)','n.(.---[mf].)','(3[2345][0-9])')
getforms(resptb,'w-stem mp','(ω|όμαι)','v(..[pia]i[ep])','(3[2345][0-9])')


In [9]:
getforms(resptb,'parts and infs','.','v(...[np]....)','(3[2345][0-9])')


In [6]:
getforms(resptb,'subj and opt','.','v(...[so].)','(3[2345][0-9])')


In [6]:
getforms(resptb,'to-do','.','lemma="(πράσσω|ποιέω|δράω)"','(3[2345][0-9])')


In [9]:
getforms(resptb,'good-beautiful','.','lemma="(καλός|ἀγαθός)"','(3[2345][0-9])')


In [21]:
getforms(resptb,'all-words','.','(postag)','(3[2345][0-9])')


In [20]:
getforms(anabtb,'all-words','.','(postag)','^1')
